In [145]:
from selenium import webdriver
import time
import os
from gsheets import Sheets
import pandas as pd
import operator
import mysql.connector
import shutil
from shutil import copyfile
import os.path
import csv
from gspread_pandas import Spread, Client 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import gspread
import math
from selenium.webdriver.chrome.options import Options

In [146]:
##removing the annoying tab that toggles and says something like: chrome is monitoring automated something something 
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [147]:
#reading credientials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#wd for authentiction file
os.chdir('/Users/innovatus6')
creds = ServiceAccountCredentials.from_json_keyfile_name('linkupdater-631ef6e77556.json', scope)

In [148]:
#getting permission 
client = gspread.authorize(creds)

In [149]:
#opening production sheet
sheet= client.open("Master Data Builds").worksheet("test1")

In [150]:
df = pd.DataFrame(sheet.get_all_records())

In [151]:
harvestfield=df[df.status == 'need'].reset_index(drop=True)

In [152]:
harvestfield

,status,name,email,number,organisation,title,country
0,need,t49,t49@gmail.com,49,im,bot49,
1,need,t50,t50@gmail.com,50,im,bot50,Narnia
2,need,t51,t51@gmail.com,51,im,bot51,Australia
3,need,t52,t52@gmail.com,,im,bot,Narna
4,need,t53,t53@gmail.com,53,im,bot53,Australia
5,need,t54,t54@gmail.com,54,im,bot54,USA
6,need,t55,t55@gmail.com,55,im,bot55,USA
7,need,t56,t56@gmail.com,56,,bot56,USA
8,need,t57,t57@gmail.com,,,,Australia
9,need,t58,t58@gmail.com,58,im,bot58,USA


In [153]:
#country dictionary
countrydictionary={
    'USA':'1',
    'Australia':'2',
    'Narnia':'3'    
}

In [154]:
web = webdriver.Chrome(chrome_options=options, executable_path='/Users/innovatus6/chromedriver')

##the below code fills in the registration page and submits it
for i in range(len(harvestfield.email)):        
    print(str(i)+" start check")
    
    try:
        web.get("https://www.bigmarker.com/innovatus-digital/Jason-He-VS-Cassius-clay")
        time.sleep(10)            

        #name
        first=harvestfield.name[i]
        firstslot=web.find_element_by_xpath('//*[@id="new_member_full_name"]')
        time.sleep(1)
        firstslot.send_keys(first)
        time.sleep(1)

        #email
        email=harvestfield.email[i]
        emailslot=web.find_element_by_xpath('//*[@id="new_member_email"]')
        time.sleep(1)
        emailslot.send_keys(email)
        time.sleep(1)

        #number
        contactnumber=harvestfield.number[i]
        contactnumberslot=web.find_element_by_xpath('//*[@id="conference_registration_pre_conference_responses_attributes_0_response"]')
        time.sleep(1)
        contactnumberslot.send_keys(str(contactnumber))
        time.sleep(1)

        #organisation
        organisation=harvestfield.organisation[i]
        organisationslot=web.find_element_by_xpath('//*[@id="conference_registration_pre_conference_responses_attributes_1_response"]')
        time.sleep(1)
        organisationslot.send_keys(organisation)
        time.sleep(1)

        #job title    
        jobtitle=harvestfield.title[i]
        jobtitleslot=web.find_element_by_xpath('//*[@id="conference_registration_pre_conference_responses_attributes_2_response"]')
        time.sleep(1)
        jobtitleslot.send_keys(jobtitle)
        time.sleep(1)


        #country
        country=int(countrydictionary[harvestfield.country[i]])
        countrybutton=web.find_element_by_xpath('//*[@id="conference_registration_pre_conference_responses_attributes_3_response"]/option['+str(country+1)+']')

        time.sleep(1)
        countrybutton.click()

        #submit 
        submitbutton=web.find_element_by_xpath('//*[@id="register_member"]/div[2]/div/div[8]/div[2]/div/span')
        submitbutton.click()
        time.sleep(2)

        #rego button
        regobutton=web.find_element_by_xpath('//*[@id="register_with_pre_response_template"]')
        regobutton.click()
        time.sleep(5)
        
    except:
        #problem case (when country input has a problem)
        sheet.update_cell(int(df[df.iloc[:,2]==harvestfield.email[i]].index.to_numpy())+2,1,"problem")
        
        continue

    try: 
        time.sleep(2)
        web.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[1]/div[2]')
        print(harvestfield.email[i]+ ' has been successfully regoed')
        
        #clear cache because the browser remembers previous regos
        web.delete_all_cookies()
        
        
        #updating pm sheet        
        sheet.update_cell(int(df[df.iloc[:,2]==harvestfield.email[i]].index.to_numpy())+2,1,"yes - regoed")
        
        continue 
        
        
    except:
                  
        #the if clause checks the existence of a string unique to the landing page of an regoed person                  
        if web.find_element_by_xpath('//*[@id="webinar-template-10-li"]/div[1]/div[2]/a').text !="":

            print(harvestfield.email[i]+ ' is already regoed')

            #updating pm sheet        
            sheet.update_cell(int(df[df.iloc[:,2]==harvestfield.email[i]].index.to_numpy())+2,1,"yes - regoed")

            #clear cache because the browser remembers previous regos
            web.delete_all_cookies()

            continue
                
            
        else: 
            
            sheet.update_cell(int(df[df.iloc[:,2]==harvestfield.email[i]].index.to_numpy())+2,1,"problem")
            continue
    


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


0 start check
1 start check
t50@gmail.com is already regoed
2 start check
t51@gmail.com is already regoed
3 start check
4 start check
t53@gmail.com is already regoed
5 start check
t54@gmail.com is already regoed
6 start check
t55@gmail.com is already regoed
7 start check
8 start check
9 start check
t58@gmail.com has been successfully regoed


In [155]:
web.find_element_by_xpath('//*[@id="webinar-template-10-li"]/div[1]/div[2]/a').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="webinar-template-10-li"]/div[1]/div[2]/a"}
  (Session info: chrome=92.0.4515.107)


In [ ]:
if "sdfsd"!="":
    print("asd")